In [1]:
import pandas as pd
from pathlib import Path

paths = Path('/common/tmp/sjackson/data/uda/signals').glob('*.parquet')
paths = filter(lambda x: int(x.stem) < 40000, paths)
paths = list(paths)
dfs = []
for p in paths:
    df_temp = pd.read_parquet(p)
    df_temp['file_path'] = str(p)
    dfs.append(df_temp)
df = pd.concat(dfs)

In [2]:
df['shot_id'] = df['file_path'].apply(lambda x: int(x.split('/')[-1].split('.')[0]))
df

,name,version,description,quality,dataset,file_path,shot_id
0,ABM_CALIB_SHOT,2,Shot used for calibration (obsolete),Not Checked,abm,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
1,ABM_CHANNEL_STATUS,2,"Failed = 0, OK = 1",Not Checked,abm,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
2,ABM_CHANNEL_TYPE,2,"Channel type (0 = poloidal, 1 = co-tangential,...",Not Checked,abm,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
3,ABM_GAIN,2,Gain of pre-amplifiers,Not Checked,abm,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
4,ABM_I-BOL,2,Incident powers (x - channel),Not Checked,abm,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
...,...,...,...,...,...,...,...
2944,XTP_V8,-1,,Not Checked,xtp,/common/tmp/sjackson/data/uda/signals/18253.pa...,18253
2945,XTP_V9,-1,,Not Checked,xtp,/common/tmp/sjackson/data/uda/signals/18253.pa...,18253
2946,XTP_VF,-1,,Not Checked,xtp,/common/tmp/sjackson/data/uda/signals/18253.pa...,18253
2947,XVS_SPRED(PIXEL),-1,,Not Checked,xvs,/common/tmp/sjackson/data/uda/signals/18253.pa...,18253


### Magnetics

In [13]:
xmb = df.loc[(df.dataset == 'xmo')]

subset = xmb.sort_values('shot_id')

# Filter the subset for specific conditions
subset = subset.loc[subset.name.str.contains('XDC_PF_F_P1', case=False)]

# Get the last shot for each 'name'
last_shot = subset[['name', 'dataset', 'shot_id']].groupby('name').last()
last_shot = last_shot.rename(columns={'shot_id': 'last_shot_id'})

first_shot = subset[['name', 'dataset', 'shot_id']].groupby('name').first()
first_shot = first_shot.rename(columns={'shot_id': 'first_shot_id'})

result = pd.concat([first_shot, last_shot], axis=1)
result = result.drop('dataset', axis=1)
result

,first_shot_id,last_shot_id
name,,
XDC_PF_F_P1,11695,28355


In [4]:
xmb

,name,version,description,quality,dataset,file_path,shot_id
2270,XMB_CC/HALO/101,-1,Centre Column Halo current measurements,Not Checked,xmb,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
2271,XMB_CC/HALO/102,-1,Centre Column Halo current measurements,Not Checked,xmb,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
2272,XMB_CC/HALO/103,-1,Centre Column Halo current measurements,Not Checked,xmb,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
2273,XMB_CC/HALO/104,-1,Centre Column Halo current measurements,Not Checked,xmb,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
2274,XMB_CC/HALO/105,-1,Centre Column Halo current measurements,Not Checked,xmb,/common/tmp/sjackson/data/uda/signals/17630.pa...,17630
...,...,...,...,...,...,...,...
2147,XMB_TAE/COIL/2,-1,,Not Checked,xmb,/common/tmp/sjackson/data/uda/signals/18253.pa...,18253
2148,XMB_TAE/COIL/3,-1,,Not Checked,xmb,/common/tmp/sjackson/data/uda/signals/18253.pa...,18253
2149,XMB_TAE/EARTH/1,-1,,Not Checked,xmb,/common/tmp/sjackson/data/uda/signals/18253.pa...,18253
2150,XMB_TAE/EARTH/2,-1,,Not Checked,xmb,/common/tmp/sjackson/data/uda/signals/18253.pa...,18253


## MAST-U

### Soft X-rays

In [24]:
soft_xrays = df.loc[(df.dataset == 'xsx') | (df.dataset == 'asx')]

In [32]:
# Sort the subset by 'shot_id'
subset = soft_xrays.sort_values('shot_id')

# Filter the subset for specific conditions
subset = subset.loc[subset.name.str.contains('CH01') & subset.name.str.contains('HCAM/U')]

# Get the last shot for each 'name'
last_shot = subset[['name', 'dataset', 'shot_id']].groupby('name').last()
last_shot = last_shot.rename(columns={'shot_id': 'last_shot_id'})

first_shot = subset[['name', 'dataset', 'shot_id']].groupby('name').first()
first_shot = first_shot.rename(columns={'shot_id': 'first_shot_id'})

result = pd.concat([first_shot, last_shot], axis=1)
result = result.drop('dataset', axis=1)

result


,first_shot_id,last_shot_id
name,,
ASX/HCAM/U/CH01,46353,49475
XSX/HCAM/U/CH01,44395,51056
XSX/HCAM/U/CH01/DATA,49904,51056


In [34]:
soft_xrays.loc[soft_xrays.shot_id < 44395]

,name,version,description,quality,dataset,file_path,uda_name,uuid,shot_id,signal_type,mds_name,format,source,file_name
